<a href="https://colab.research.google.com/github/bigboom1253/e_commerce_recommendation_system_development/blob/master/Data_preprocessing/Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud
from collections import Counter
import datetime

plt.rc('font', family='NanumBarunGothic')

In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:
path = "/content/drive/My Drive/최강1조-추천 시스템/데이터 수집/"

# prod = pd.read_json(path + "all_prod_data.json")
review = pd.read_json(path + "all_review_data.json")
# satisfaction = pd.read_json(path + "all_satisfaction_data.json")

# Review

In [142]:
from datetime import datetime

def to_datetime(x):
    if x != "": return datetime.strptime(x[:10], "%Y-%m-%d")

review["review_date"] = review.review_date.apply(lambda x: to_datetime(x)) # 전처리 - datetime 형태 변환
review = review.merge(prod[['prod_idx', 'category_num', 'title', 'small_category']]) # 전처리 - 카테고리 넘버 merge 시켜줌

In [284]:
review = review.dropna()

In [285]:
word = review[['content', 'prod_satis', 'ship_satis']]
word

,content,prod_satis,ship_satis
0,생각보다 조금 얇지만 맛있게 먹을게요,4,5
1,배송빠르고 맛도 괜찮아요.,4,5
2,달지도 않고 퍽퍽...,1,3
3,맛있어요 ㅋㅋㅋ 저렴해요,5,4
4,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5
...,...,...,...
1695505,포장은 꼼꼼한데 냉동이 왔어요,1,5
1695507,제가 먹어본건 아니구요 명절선물로 롯데 백화점에서진열상품확인후 온라인배송했습니다가...,4,4
1695510,맛있게 잘먹어서 다시 구매할 예정입니다,5,5
1695511,잘 받았어요.......,5,5


In [286]:
word['length'] = word.content.apply(len)
word

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,content,prod_satis,ship_satis,length
0,생각보다 조금 얇지만 맛있게 먹을게요,4,5,20
1,배송빠르고 맛도 괜찮아요.,4,5,15
2,달지도 않고 퍽퍽...,1,3,13
3,맛있어요 ㅋㅋㅋ 저렴해요,5,4,13
4,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5,37
...,...,...,...,...
1695505,포장은 꼼꼼한데 냉동이 왔어요,1,5,17
1695507,제가 먹어본건 아니구요 명절선물로 롯데 백화점에서진열상품확인후 온라인배송했습니다가...,4,4,92
1695510,맛있게 잘먹어서 다시 구매할 예정입니다,5,5,21
1695511,잘 받았어요.......,5,5,13


In [357]:
def clean(x):
    text = re.sub('[-=_+,#\?;:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》☆★×♡♥·ᆢ\/]', '', x) # 특수문자 제거
    text = re.sub('[a-zA-Z]', '', text) # 영어 제거
    text = re.sub('[ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ]', '', text) # 모음 제거
    text = re.sub('[ㄱ-ㅎ]*', '', text) # 자음 반복 제거
    text = re.sub('[0-9]{3,}', '', text) # 숫자 3번 이상 나오면 제거
    return text

In [358]:
word['clean'] = word.content.apply(clean)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [359]:
word['length_clean'] = word.clean.apply(len)
word

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,content,prod_satis,ship_satis,length,clean,length_clean
0,생각보다 조금 얇지만 맛있게 먹을게요,4,5,20,생각보다 조금 얇지만 맛있게 먹을게요,20
1,배송빠르고 맛도 괜찮아요.,4,5,15,배송빠르고 맛도 괜찮아요,14
2,달지도 않고 퍽퍽...,1,3,13,달지도 않고 퍽퍽,10
3,맛있어요 ㅋㅋㅋ 저렴해요,5,4,13,맛있어요 저렴해요,10
4,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5,37,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,37
...,...,...,...,...,...,...
1695505,포장은 꼼꼼한데 냉동이 왔어요,1,5,17,포장은 꼼꼼한데 냉동이 왔어요,17
1695507,제가 먹어본건 아니구요 명절선물로 롯데 백화점에서진열상품확인후 온라인배송했습니다가...,4,4,92,제가 먹어본건 아니구요 명절선물로 롯데 백화점에서진열상품확인후 온라인배송했습니다가...,92
1695510,맛있게 잘먹어서 다시 구매할 예정입니다,5,5,21,맛있게 잘먹어서 다시 구매할 예정입니다,21
1695511,잘 받았어요.......,5,5,13,잘 받았어요,6


In [360]:
word[word.length_clean == 0]

,content,prod_satis,ship_satis,length,clean,length_clean
33,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,1,1,12,,0
329,good!~~~~~~~,5,5,12,,0
514,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,1,1,12,,0
635,ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ,5,5,11,,0
780,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,5,5,13,,0
...,...,...,...,...,...,...
1694539,ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ,5,5,10,,0
1694541,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,5,5,13,,0
1694914,",,,,,,,,,,",5,5,10,,0
1694961,ㄱㄴㄱㄴㄱㄴㄱㄴㄱㄴ,4,4,10,,0


In [361]:
word[word.length_clean == 1]

,content,prod_satis,ship_satis,length,clean,length_clean
929,gòooooooooooood,5,4,15,ò,1
1393,ㅎㅎㅎㅎㅎㅎㅎㅎ호ㅗㅗㅗ,4,4,12,호,1
1455,굿~~~~~~~~~~~,5,4,12,굿,1
2293,op;''/ jkioo,5,4,12,,1
2750,굿~~~~~~^^^^,4,4,11,굿,1
...,...,...,...,...,...,...
1693823,굳~~~~~~~~^~~,5,5,12,굳,1
1693960,ㄷㅅㄴㅅㆍㅅㆍㅅㄷㅅㄷㅅㆍㅅ,4,5,15,,1
1694944,굿!!!!!!!!!,5,5,10,굿,1
1694945,굿!!!!!!!!!,5,5,10,굿,1


In [362]:
word[word.length_clean == 2]

,content,prod_satis,ship_satis,length,clean,length_clean
109,굿굿!!!!!!!!,3,3,10,굿굿,2
129,굿굿!!!!!!!!!,4,4,11,굿굿,2
1681,ㄱㄹㅁ읏ㄹㄴ낙ㄱㅁㅎ,2,2,10,읏낙,2
1943,very delicious,5,5,15,,2
2516,별로..........,1,1,12,별로,2
...,...,...,...,...,...,...
1689814,Delicious and good,5,5,18,,2
1689837,Delicious and good,5,5,18,,2
1691323,ㄷㅈ늑ㅂㅈㅇㅇㅇㅇ으ㅇㅈㅅ,4,2,13,늑으,2
1691873,Xbnyek74krvjfymrumt,4,4,19,74,2


In [363]:
np.unique(word[word.length_clean == 3].clean)

array(['   ', '  3', '  6', '  7', '  ù', '  굿', '  머', '  흗', ' 55',
       ' 76', ' 9 ', ' 븍지', ' 실망', ' 쓰 ', ' 조앙', ' 존맛', ' 츄 ', '0제즈',
       '1  ', '12호', '2긔니', '2아요', '4”’', '5  ', '6듢든', '6튜푸', '7 8',
       '77퍼', '89 ', 'û55', 'ᆞᆞᆞ', 'ᆞ으응', 'Ỳ67', '⁷  ', '가가가', '가격짱',
       '가뒧두', '가성비', '간쥑뉘', '감사 ', '감사링', '감사요', '갓순필', '강추요', '개꿀맛',
       '개존맛', '갡디늣', '갸어내', '고찮음', '곻고솦', '괘찮음', '괜찮길', '괜찮음', '괜춘요',
       '구 굿', '구8스', '구국국', '구숙궄', '구웃웃', '국굿굿', '굳  ', '굳 디', '굳 횻',
       '굳11', '굳굳 ', '굳굳굳', '굳밤 ', '굳짗짖', '굿  ', '굿굯근', '굿굿 ', '굿굿굿',
       '굿뜨요', '굿이요', '굿이쥬', '굿잡 ', '굿푸짐', '굿호호', '규규규', '그그긪', '그냥 ',
       '그냥요', '그니특', '극그극', '극긔극', '극니그', '극브븝', '극지짇', '근굿굿', '근너느',
       '근버는', '글그는', '급급긋', '급듯7', '긋  ', '긋긋 ', '긋니싲', '긔니그', '긘득그',
       '긪■∫', '기  ', '기기읟', '기너우', '기보너', '기보노', '기보니', '기보련', '기보보',
       '기본마', '깁낍핀', '깁읃빕', '꿀맛탱', '나서 ', '내내넌', '냄새가', '냐채텩', '너나마',
       '너난넌', '너눙5', '너마머', '너웅니', '넘맛 ', '넘작다', '네여 ', '노가가', '노녹4',
       '녹그시', '녹아서',

In [364]:
word[word.length_clean == 4]

,content,prod_satis,ship_satis,length,clean,length_clean
119,괜찮아요~~~~~~~~~,4,4,13,괜찮아요,4
157,좋아요 ♡♡♡♡♡♡♡♡♡,5,5,13,좋아요,4
167,짱이에요!!!!!!!!!,5,5,13,짱이에요,4
724,맛있어요~~~~~~~~~~~~~~~~~,5,5,21,맛있어요,4
904,그냥저냥~~~~~~~~~~,4,4,14,그냥저냥,4
...,...,...,...,...,...,...
1694596,맛있어요~~~~~~~,5,5,11,맛있어요,4
1694671,맛있어여~~~~~~~,5,5,11,맛있어여,4
1695089,맛있어요~~~~~~,5,5,10,맛있어요,4
1695303,맛있어요~~~~~%,5,5,10,맛있어요,4


In [365]:
word[word.length_clean == 5]

,content,prod_satis,ship_satis,length,clean,length_clean
219,좋아요 굿!!!!!ㅎㅎ,5,5,12,좋아요 굿,5
662,양호합니다~~~~~~~~~,4,4,14,양호합니다,5
1049,만족합니다~~~!!,3,3,10,만족합니다,5
1064,감사드려요!!!!!!!!,5,5,13,감사드려요,5
1340,보통입니다.....,3,4,10,보통입니다,5
...,...,...,...,...,...,...
1694329,맛있어요 ㅎㅎㅎㅎㅎㅎ,5,5,11,맛있어요,5
1694387,잘먹을께요......,4,5,11,잘먹을께요,5
1694782,맛있어요 ^^^^#####,5,5,14,맛있어요,5
1695130,늘 시켜요......,5,5,11,늘 시켜요,5


In [366]:
word[word.length_clean == 6]

,content,prod_satis,ship_satis,length,clean,length_clean
62,잘 먹었어요~~~~~~~~~~~~~,4,4,19,잘 먹었어요,6
120,좋아요ㅎㅎ굳굳굳ㅎㅎ,5,5,10,좋아요굳굳굳,6
207,잘받았습니당ㅎㅎㅎㅎ,5,5,10,잘받았습니당,6
463,밤고구마네여ㅜㅜㅜㅜㅜㅜ,3,2,12,밤고구마네여,6
739,잘받았습니다★★★★,4,4,10,잘받았습니다,6
...,...,...,...,...,...,...
1693564,너무 맛나요~~~~~,5,5,11,너무 맛나요,6
1693696,잘받았습니다~~~~~,5,5,11,잘받았습니다,6
1694569,잘받았습니다~~~~~,4,4,11,잘받았습니다,6
1695358,"그냥 그래요,,,,,",3,5,11,그냥 그래요,6


## 10자 미만인 것들은 리뷰에 0.5정도를 곱?

In [370]:
len(word[word.length_clean < 10]) / len(word) * 100

6.742947817936441

In [368]:
word[word.length_clean > 10]

,content,prod_satis,ship_satis,length,clean,length_clean
0,생각보다 조금 얇지만 맛있게 먹을게요,4,5,20,생각보다 조금 얇지만 맛있게 먹을게요,20
1,배송빠르고 맛도 괜찮아요.,4,5,15,배송빠르고 맛도 괜찮아요,14
4,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5,37,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,37
5,며칠동안 놔뒀다가 먹으니까 되게 맛있네요 다음에 또 시킬게요!!!!,5,5,37,며칠동안 놔뒀다가 먹으니까 되게 맛있네요 다음에 또 시킬게요,33
6,좀 실망스럽지만 잘 먹었어요,3,2,15,좀 실망스럽지만 잘 먹었어요,15
...,...,...,...,...,...,...
1695504,다음에 또 주문 하겠습니다,5,5,14,다음에 또 주문 하겠습니다,14
1695505,포장은 꼼꼼한데 냉동이 왔어요,1,5,17,포장은 꼼꼼한데 냉동이 왔어요,17
1695507,제가 먹어본건 아니구요 명절선물로 롯데 백화점에서진열상품확인후 온라인배송했습니다가...,4,4,92,제가 먹어본건 아니구요 명절선물로 롯데 백화점에서진열상품확인후 온라인배송했습니다가...,92
1695510,맛있게 잘먹어서 다시 구매할 예정입니다,5,5,21,맛있게 잘먹어서 다시 구매할 예정입니다,21
